In [7]:
using Test

In [25]:
using FeedbackParticleFilters, Distributions, Random, PDMats, LinearAlgebra, Statistics

In [3]:
f(x::AbstractVector) = [x[1]*x[2], x[1]-x[3], x[1]+x[2]*x[3]]
g(x::AbstractVector) = [x[1] x[2]; x[2] x[3]; 0. 1.]
init                 = MvNormal(3, 1.)

mod = DiffusionStateModel(f, g, init)

Diffusion process model for the hidden state
    type of hidden state:                   3-dimensional vector
    number of independent Brownian motions: 2
    initial condition:                      random

In [8]:
@test mod isa DiffusionStateModel{Float64,typeof(f),typeof(g),MvNormal{Float64,PDMats.ScalMat{Float64},Distributions.ZeroVector{Float64}}}

Test Passed

In [11]:
N = 6
    x_pf=[-2.422480820086937 -0.592332203303167 -2.017301296096984 -1.5151245392598531 0.02565906919199346 0.15161614796874012;]
    testens=UnweightedParticleEnsemble(x_pf)
    eq = PoissonEquation(x->[x[1],exp(x[1])], testens)

Poisson equation for the gain
    # of particles:        6
    hidden dimension:      1
    observed dimension:    2

In [15]:
 update!(eq, testens)

Poisson equation for the gain
    # of particles:        6
    hidden dimension:      1
    observed dimension:    2

In [16]:
eq.gain

1×6×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0

In [26]:
# Helper functions for DifferentialRKHSMethod
function DifferentialRKHS_MakeMatrices(vec::AbstractVector, eps::Number)
    # constructs kernel matrices for DifferentialRKHSMethod
    L = length(vec)
    eps1 = 2*eps
    eps2 = eps^2

    mat = zeros(Float64, L, L, 3)

    @inbounds @simd for i in 1:L
                mat[i,i,1] = 1
                mat[i,i,3] = 1/eps
                @simd for j in i+1:L
                    diff = vec[j] - vec[i]
                    sq   = diff*diff
                    K    = exp(-sq/eps1)
                    Kx   = diff*K/eps
                    Kxy  = (eps - sq) * K / eps2
                    mat[i,j,1] = mat[j,i,1] = K
                    mat[i,j,2] = Kx
                    mat[j,i,2] = -Kx
                    mat[i,j,3] = mat[j,i,3] = Kxy
                end
    end

    return view(mat, :, :, 1), view(mat, :, :, 2), view(mat, :, :, 3)
end



function DifferentialRKHS_M_and_b!(A::AbstractMatrix, B::AbstractMatrix, C::AbstractMatrix, v::AbstractMatrix, lamb::Number)
    # constructs matrix M and vector b for DifferentialRKHSMethod
    L = size(A,1)
    m = size(v,2)

    b = zeros(Float64, 2*L, m)
    LinearAlgebra.mul!(view(b,   1:L  , :), A, v)
    LinearAlgebra.mul!(view(b, L+1:2*L, :), B, v)


    B2 = B*B
    BC = B*C
    C2 = C*C
    LinearAlgebra.lmul!(lamb, A)
    LinearAlgebra.lmul!(lamb, B)
    LinearAlgebra.lmul!(lamb, C)

    M = zeros(Float64, 2*L, 2*L)
    @simd for j in 1:L
        @simd for i in 1:L
            @inbounds M[i  ,j  ] =  A[i,j] - B2[i,j]
            @inbounds M[i+L,j  ] =  B[i,j] + BC[j,i]
            @inbounds M[i  ,j+L] = -B[i,j] - BC[i,j]
            @inbounds M[i+L,j+L] =  C[i,j] + C2[i,j]
        end
    end

    return M, b
end
function solve2!(eq::PoissonEquation, method::DifferentialRKHSMethod)
    eps  = method.epsilon
    N    = no_of_particles(eq)
    lamb = N * method.lambda

    H = Htilde(eq)

    # Gaussian kernel and partial derivative matrices
    K, Kx, Kxy = DifferentialRKHS_MakeMatrices(eq.positions[1,:], eps)

    # compute M and b
    M, b = DifferentialRKHS_M_and_b!(K, Kx, Kxy, H', lamb) # warning: this function multiplies K, Kx, and Kxy by lamb

    # solve linear system
    beta = M \ b

    # write potential and gain
    eq.potential  .= ( K*view(beta, 1:N) - Kx*view(beta, N+1:2*N) )' / lamb
    broadcast!(-, eq.potential, eq.potential, Statistics.mean(eq.potential, dims=2))
    eq.gain[1,:,:] = ( Kx * view(beta, 1:N, :) + Kxy * view(beta, N+1:2*N, :) ) / lamb
end

solve2! (generic function with 1 method)

In [27]:
solve2!(eq, DifferentialRKHSMethod(1E1, 1E-6));

In [1]:
function Map(f::T, A::AbstractArray) where T<:Function 
    map(f, A)
end
function Map(F::NTuple{N, Function}, x::T) where {N, T<:Number} 
    [f(x) for f in F]
end;
function Map(F::AbstractArray{Function}, x::T) where {N, T<:Number} 
    [f(x) for f in F]
end;
function Map(F::AbstractArray{U}, x::T) where {N, T<:Number, U<:Function}
    [f(x) for f in F]
end;
function Map(F::NTuple{N, Function}, A::AbstractArray; output_shape=2) where N
    if output_shape == 1 
        try
            collect(Map.(F, Ref(A)))
        catch
            error("ERROR: functions cannot be applied at first level. Call with output_shape=2.")
        end
    elseif output_shape == 2
        try
            [f(A) for f in F]
        catch
            collect(Map.(Ref(F), A))
        end
    else
        error("ERROR: Invalid output_shape parameter. Must be either 1 or 2.") 
    end
end;
function Map(F::AbstractArray{T}, A::AbstractArray; output_shape=2) where {N, T<:Function}
    if output_shape == 1 
        try
            collect(Map.(F, Ref(A)))
        catch
            error("ERROR: functions cannot be applied at first level. Call with output_shape=2.")
        end
    elseif output_shape == 2
        try
            [f(A) for f in F]
        catch
            collect(Map.(Ref(F), A))
        end
    else
        error("ERROR: Invalid output_shape parameter. Must be either 1 or 2.") 
    end
end;

In [2]:
f(x::Float64) = x
g(x::Float64) = x^2
h(x::Float64) = x^3;

In [42]:
Map((f,g,h), [[1.,2.],[3.,4.]], output_shape=2) == [[[1.0, 1.0, 1.0], [2.0, 4.0, 8.0]], [[3.0, 9.0, 27.0], [4.0, 16.0, 64.0]]]

true

In [46]:
ff(x::Array{Float64,1}) = x[1]*x[2]
gg(x::Array{Float64,1}) = x[1]+x[2]
hh(x::Array{Float64,1}) = x[1]-x[2];

In [52]:
Map((ff,gg,hh), [[1.,2.],[3.,4.]], output_shape=2) == [[2.0, 3.0, -1.0], [12.0, 7.0, -1.0]]

true